<a href="https://colab.research.google.com/github/PedroTechy/DataProcessingEdit/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregations
- Group By
- Windows Functions

# Setting up PySpark

In [1]:
# @title
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()

# Aggregations

https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions

https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-aggregate.html

In [33]:
sql_query = """CREATE OR REPLACE TEMPORARY VIEW basic_pays AS SELECT * FROM VALUES
('Diane Murphy','Accounting',8435),
('Mary Patterson','Accounting',9998),
('Jeff Firrelli','Accounting',8992),
('William Patterson','Accounting',8870),
('Gerard Bondur','Accounting',11472),
('Anthony Bow','Accounting',6627),
('Leslie Jennings','IT',8113),
('Leslie Thompson','IT',5186),
('Julie Firrelli','Sales',9181),
('Steve Patterson','Sales',9441),
('Foon Yue Tseng','Sales',6660),
('George Vanauf','Sales',10563),
('Loui Bondur','SCM',10449),
('Gerard Hernandez','SCM',6949),
('Pamela Castillo','SCM',11303),
('Larry Bott','SCM',11798),
('Barry Jones','SCM',10586)
AS basic_pays(employee_name, department, salary)"""

# creating temp view
spark.sql(sql_query)

df = spark.table("basic_pays")
df.show()


+-----------------+----------+------+
|    employee_name|department|salary|
+-----------------+----------+------+
|     Diane Murphy|Accounting|  8435|
|   Mary Patterson|Accounting|  9998|
|    Jeff Firrelli|Accounting|  8992|
|William Patterson|Accounting|  8870|
|    Gerard Bondur|Accounting| 11472|
|      Anthony Bow|Accounting|  6627|
|  Leslie Jennings|        IT|  8113|
|  Leslie Thompson|        IT|  5186|
|   Julie Firrelli|     Sales|  9181|
|  Steve Patterson|     Sales|  9441|
|   Foon Yue Tseng|     Sales|  6660|
|    George Vanauf|     Sales| 10563|
|      Loui Bondur|       SCM| 10449|
| Gerard Hernandez|       SCM|  6949|
|  Pamela Castillo|       SCM| 11303|
|       Larry Bott|       SCM| 11798|
|      Barry Jones|       SCM| 10586|
+-----------------+----------+------+



In [ ]:
perc_query = """SELECT
    department,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) AS pc1,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pc2,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pc3,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pc4,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) AS pd1,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pd2,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pd3,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pd4
FROM basic_pays
GROUP BY department
ORDER BY department;"""

spark.sql(perc_query).show()

+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|department|    pc1|     pc2|    pc3|     pc4|    pd1|    pd2|    pd3|    pd4|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|Accounting|8543.75| 7838.25| 9746.5|10260.75| 8435.0| 6627.0| 9998.0|11472.0|
|        IT|5917.75|    NULL|7381.25|    NULL| 5186.0|   NULL| 8113.0|   NULL|
|       SCM|10449.0|10786.25|11303.0|11460.75|10449.0|10449.0|11303.0|11798.0|
|     Sales|8550.75|    NULL| 9721.5|    NULL| 6660.0|   NULL|10563.0|   NULL|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+



In [ ]:
from pyspark.sql.functions import *
(df
 .groupBy("department")
 .agg(sum("salary").alias("sum_salary"),
      round(avg("salary"),2).alias("avg_salary"),
      min("salary").alias("min_salary"),
      array_agg("employee_name").alias("employees"),
      count(lit("")).alias("count_employees"))
 .filter(col("count_employees") > 2)
 .show(10, False))

+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|department|sum_salary|avg_salary|min_salary|employees                                                                                   |count_employees|
+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|Sales     |35845     |8961.25   |6660      |[Julie Firrelli, Steve Patterson, Foon Yue Tseng, George Vanauf]                            |4              |
|Accounting|54394     |9065.67   |6627      |[Diane Murphy, Mary Patterson, Jeff Firrelli, William Patterson, Gerard Bondur, Anthony Bow]|6              |
|SCM       |51085     |10217.0   |6949      |[Loui Bondur, Gerard Hernandez, Pamela Castillo, Larry Bott, Barry Jones]                   |5              |
+----------+----------+----------+----------+-------------------------

# Question

In [36]:
# Q1
# Aggregate data by surname
# Calculate highest salary by surname
# Include the respective employee that has the highest salary
# Include department information about this employee
# Count how many employees has that surname
# Put in an array all the first_names of the respective surname ordered


from pyspark.sql import functions as F

#Added column "surname" to my dataframe based on the col emplyoee_name
df = df.withColumn("surname", split(col("employee_name"), " ")[1])
df = df.withColumn("first_name", split(col("employee_name"), " ")[0])

df_temp = df.groupBy("surname").agg(
    F.max("salary").alias("max_salary_by_surname"),
    F.count(lit(" ")).alias("count_employees_for_surname"),
    F.array_agg("first_name").alias("first_names"),
    filter(col("first_names"))
)
df_temp.show()
















# schema expected:
# surname | count_employees | highest_salary | employee_with_highest_salary | department_with_highest_salary | array_with_all_the_first_names |

+---------+---------------------+---------------------------+--------------------+
|  surname|max_salary_by_surname|count_employees_for_surname|          firt_names|
+---------+---------------------+---------------------------+--------------------+
|      Bow|                 6627|                          1|           [Anthony]|
|    Jones|                10586|                          1|             [Barry]|
|   Bondur|                11472|                          2|      [Gerard, Loui]|
|   Murphy|                 8435|                          1|             [Diane]|
| Castillo|                11303|                          1|            [Pamela]|
| Firrelli|                 9181|                          2|       [Jeff, Julie]|
|   Vanauf|                10563|                          1|            [George]|
|      Yue|                 6660|                          1|              [Foon]|
|Patterson|                 9998|                          3|[Mary, William, S...|
| Th

In [ ]:


from pyspark.sql import functions as F

# Split the employee_name into first_name and surname
df = df.withColumn("first_name", split(col("employee_name"), " ").getItem(0)) \
       .withColumn("surname", split(col("employee_name"), " ").getItem(1))

# Get the highest salary for each surname and the employee with the highest salary
window_spec = Window.partitionBy("surname").orderBy(F.desc("salary"))

df_with_highest = df.withColumn("row_number", F.row_number().over(window_spec)) \
                    .filter(F.col("row_number") == 1) \
                    .select("surname", "salary", "employee_name", "department") \
                    .withColumnRenamed("salary", "highest_salary") \
                    .withColumnRenamed("employee_name", "employee_with_highest_salary") \
                    .withColumnRenamed("department", "department_with_highest_salary")

# Aggregate data by surname
result = df.groupBy("surname") \
           .agg(F.count("surname").alias("count_employees"),
                F.collect_list("first_name").alias("first_names")) \
           .join(df_with_highest, on="surname") \
           .withColumn("array_with_all_the_first_names", F.expr("sort_array(first_names)"))

# Select and order the final columns
result.select("surname",
              "count_employees",
              "highest_salary",
              "employee_with_highest_salary",
              "department_with_highest_salary",
              "array_with_all_the_first_names").show(truncate=False)
